<a href="https://colab.research.google.com/github/j-ranasinghe/Spam-E-mail-Detection/blob/main/Spam_E_mail_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns

# Importing the dataset

In [ ]:
cols = ['capital_run_length_total','capital_run_length_longest','capital_run_length_average','char_freq_#','char_freq_$','char_freq_!','char_freq_[','char_freq_(','char_freq_;','word_freq_conference','word_freq_table','word_freq_edu','word_freq_re','word_freq_project','word_freq_original','word_freq_meeting','word_freq_cs','word_freq_direct','word_freq_pm','word_freq_parts','word_freq_1999','word_freq_technology','word_freq_85','word_freq_415','word_freq_data','word_freq_857','word_freq_telnet','word_freq_labs','word_freq_lab','word_freq_650','word_freq_george','word_freq_hpl','word_freq_hp','word_freq_money','word_freq_000','word_freq_font','word_freq_your','word_freq_credit','word_freq_you','word_freq_email','word_freq_business','word_freq_free','word_freq_addresses','word_freq_report','word_freq_people','word_freq_will','word_freq_receive','word_freq_mail','word_freq_order','word_freq_internet','word_freq_remove','word_freq_over','word_freq_our','word_freq_3d','word_freq_all','word_freq_address','word_freq_make','class']
df = pd.read_csv('spambase.data' , names= cols )
#d = preprocessing.normalize(df, axis = 1)
#scaled_df = pd.DataFrame(d, columns=names)
df.head()

In [ ]:
df.head(20)

In [ ]:
df.tail(10)

# Data Visualisation

In [ ]:
df.dtypes #Check the data type of the attributes


In [ ]:
df.shape #Check the shape of the dataset

In [ ]:
#Check the distribution of the spam vs not-spam 
new_class = pd.Categorical(df["class"])
new_class = new_class.rename_categories(["spam","not_spam"])              

new_class.describe()

In [ ]:
df.describe() #Get the descriptions of each attribute

In [ ]:
df.isnull()

In [ ]:
#Boxplot to for outliers
for label in cols[:-1]:
    # for label in range(0,5):
    df.plot(
    kind='box', 
    subplots=True, 
    sharey=False, 
    figsize=(70, 25)
    )
        # increase spacing between subplots
    plt.subplots_adjust(wspace=0.9) 
    plt.show()
    break

In [ ]:
for label in cols[:-1]:
  plt.hist(df[df["class"]==1][label],color= 'blue',label='spam',alpha=0.5,density = True)
  plt.hist(df[df["class"]==0][label],color= 'red',label='non-spam',alpha=0.5,density = True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel("label")
  plt.show()


In [ ]:
X = df.drop(['capital_run_length_total','word_freq_project','word_freq_conference','capital_run_length_longest','char_freq_#','word_freq_remove','word_freq_3d','word_freq_receive','word_freq_mail',], axis = 1)

In [ ]:
X.head(12)


In [ ]:
X.tail(12)